In [4]:
!pip install kfp --upgrade

     |████████████████████████████████| 224 kB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 14.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 34 kB/s s eta 0:00:01
     |████████████████████████████████| 54 kB 3.6 MB/s  eta 0:00:01
     |████████████████████████████████| 52 kB 1.9 MB/s  eta 0:00:01
     |████████████████████████████████| 82 kB 1.3 MB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 8.3 MB/s  eta 0:00:01
  Created wheel for kfp: filename=kfp-1.6.6-py3-none-any.whl size=308915 sha256=091c559694044c8c4991696092f56b511d4e4d3916f57c64e38c1b5cb75d28dd
  Stored in directory: /home/jupyter/.cache/pip/wheels/22/ec/07/d8a332e22c1098a45bdd0c54cb0dafd84bad0ca0e4f1b734ae
  Created wheel for docstring-parser: filename=docstring_parser-0.9.

In [15]:
import subprocess

In [5]:
import kfp
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics)

from kfp.v2 import compiler

from pipeline.preproc_split_dataset import preproc_split_dataset
from pipeline.train import train_model
from pipeline.evaluate_model import evaluate_model
#from helper_components import build_image
#from helper_components import custom_deploy

In [1]:
REGION = 'us-central1'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [23]:
# ativar Cloud Resource Manager API
project_number= subprocess.run(['gcloud', 'projects', 'describe', PROJECT_ID, '--format', 'value(projectNumber)'],
               capture_output=True).stdout.decode('UTF-8').replace('\n','')
sa= '{}-compute@developer.gserviceaccount.com'.format(project_number)
sa

'383073583641-compute@developer.gserviceaccount.com'

In [2]:
!gsutil mb -p $PROJECT_ID  gs://kubeflow-lab-v2
!gsutil ls

Creating gs://kubeflow-lab-v2/...
gs://artifacts.rugged-shell-320613.appspot.com/
gs://hp-tuning-file/
gs://kubeflow-lab-v2/
gs://rugged-shell-320613_cloudbuild/


In [24]:
!gsutil iam ch serviceAccount:$sa:objectAdmin gs://kubeflow-lab-v2/

In [6]:
root_path = "gs://kubeflow-lab-v2"

@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=root_path ,
    # A name for the pipeline. Use to determine the pipeline Context.
    name="pipeline-test-1")


def pipeline(alpha_val:float=0.001,max_iter_val:int=100,root_path:str=root_path):
    dataset_op = preproc_split_dataset(root_path=root_path)
    train_op = train_model(dataset_train=dataset_op.outputs["dataset_train"],
                           dataset_validation=dataset_op.outputs["dataset_validation"],
                           alpha=alpha_val,
                           max_iter=max_iter_val
                          )
    eval_op = evaluate_model(
        test_set=dataset_op.outputs["dataset_test"],
        model_artifact=train_op.outputs["model_artifact"]
    )

In [7]:
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='sklearn_pipe.json')